<br>

# Compute Salinity Index
***

In [1]:
import xarray as xr
import scipy.signal as sg
import numpy as np

<br>

### Define Funtions
***

In [2]:
def centerlon(data):
    i = data.lon
    data = xr.concat([
        data.sel(lon=i[i >= 180]),
        data.sel(lon=i[i <  180])],
        dim="lon"
    )
    data = data.assign_coords({"lon": (((data.lon + 180) % 360) - 180)})

    return data

In [3]:
def timeplace(data, lat_s, lat_e, lon_s, lon_e, year_s, year_e):
    data = data.sel(                              
        lat = slice(lat_s,lat_e),
        lon = slice(lon_s,lon_e),
        time = (data["time.year"] >= year_s)
             & (data["time.year"] <= year_e),
    )
    return data

In [4]:
def detrend_anoms(data):

    
    ### Substract Climatology
    
    anom_d = (data
        .groupby("time.month")
        .mean(("time"))
    )
    anom_d = data.groupby('time.month') - anom_d
    
    
    ### Detrend Data
    
    for lat in range(len(anom_d.lat)):
        for lon in range(len(anom_d.lon)):

            if not xr.ufuncs.isnan(anom_d[:, lat, lon]).sum().isin([0, len(anom_d.time)]):
                anom_d[:, lat, lon] = 'nan'

            elif xr.ufuncs.isnan(anom_d[:, lat, lon]).sum().isin([0]):
                anom_d[:, lat, lon] = sg.detrend(anom_d[:, lat, lon], axis=0)

    
    return anom_d

In [5]:
def seasonal_mean(data, month_s, month_e):
    
    data = (
        data.sel(
            time=(data["time.month"] >= month_s)
            & (data["time.month"] <= month_e)
        )
        .groupby('time.year')
        .mean('time')
    )
    
    return data

<br>

### Set Years
---

In [6]:
year_s = 1901
year_e = 2017

<br>

## Salinity
---
---

### Set Region and Month

In [7]:
lat_s = -40
lat_e = 60

lon_s = -60
lon_e = 50

month_s = 3  # April
month_e = 5  # June

### Import Data

In [8]:
sssa_d = xr.open_dataset('data/da_my_sss.nc').salinity
# sssa_d

### Process Data

In [9]:
# %%time
### Center around 0-meridian
sssa_d = centerlon(sssa_d)

### Choose Time Range and Region
sssa_d = timeplace(sssa_d, lat_s, lat_e, lon_s, lon_e, year_s, year_e)


### Compute Anomalies and Detrend them
sssa_d = detrend_anoms(sssa_d)

### Seasonal Mean
sssa_d = seasonal_mean(sssa_d, month_s, month_e)

sssa_d

<xarray.DataArray 'salinity' (year: 117, lat: 101, lon: 111)>
array([[[-1.29483761e-02, -1.79767013e-02, -2.45899782e-02, ...,
          8.24898407e-02,  8.23295116e-02,  8.75569209e-02],
        [-5.99188497e-03, -1.05279656e-02, -1.65572166e-02, ...,
          7.90075660e-02,  7.93530717e-02,  8.39161575e-02],
        [            nan,             nan,  9.28053632e-04, ...,
          6.08513393e-02,  6.18646741e-02,  6.61503524e-02],
        ...,
        [ 1.53978588e-02,  2.04161461e-02,  2.70861685e-02, ...,
                     nan,             nan,             nan],
        [ 2.24857405e-02,  2.87845414e-02,  3.72157432e-02, ...,
                     nan,             nan,             nan],
        [ 3.48514654e-02,  4.22592908e-02,  5.20099290e-02, ...,
                     nan,             nan,             nan]],

       [[-1.26138581e-02, -1.73650663e-02, -2.36428622e-02, ...,
          7.80608356e-02,  7.75573328e-02,  8.21818411e-02],
        [-6.12811418e-03, -1.04124583e-02, -1.61131248e-02, ...,
          7.52153024e-02,  7.52948523e-02,  7.93757439e-02],
        [            nan,             nan,  7.95959204e-04, ...,
          5.79922386e-02,  5.87892421e-02,  6.26807138e-02],
...
                     nan,             nan,             nan],
        [ 4.36918773e-02,  2.87430529e-02,  1.67012755e-02, ...,
                     nan,             nan,             nan],
        [-1.55900174e-03, -1.43310502e-02, -2.44992431e-02, ...,
                     nan,             nan,             nan]],

       [[-8.42083454e-01, -8.58600914e-01, -8.65347326e-01, ...,
          1.07957572e-02,  1.08452467e-02,  3.38234082e-02],
        [-8.18737447e-01, -8.32812488e-01, -8.33167851e-01, ...,
         -4.83304774e-03,  1.65346509e-03,  2.58910339e-02],
        [            nan,             nan, -6.21503413e-01, ...,
         -7.11249709e-02, -6.20889664e-02, -3.79112773e-02],
        ...,
        [ 2.53602862e-01,  2.28916302e-01,  1.98383614e-01, ...,
                     nan,             nan,             nan],
        [ 1.23260908e-01,  8.45848098e-02,  3.91124077e-02, ...,
                     nan,             nan,             nan],
        [-2.98561826e-02, -7.97280148e-02, -1.37430370e-01, ...,
                     nan,             nan,             nan]]],
      dtype=float32)
Coordinates:
    depth    float32 5.022
  * lat      (lat) float32 -40.0 -39.0 -38.0 -37.0 -36.0 ... 57.0 58.0 59.0 60.0
  * lon      (lon) float64 -60.0 -59.0 -58.0 -57.0 -56.0 ... 47.0 48.0 49.0 50.0
  * year     (year) int64 1901 1902 1903 1904 1905 ... 2013 2014 2015 2016 2017

<br>

## Box Indices (as suggested in Li2016)
---

In [10]:
def reg_mean(data, region, nanregion):

    data.loc[
        :,
        region_nan['lat_s']:region_nan['lat_e'],
        region_nan['lon_s']:region_nan['lon_e']
    ] = np.nan
    
    data = (
        data.sel(
            lat=slice(region['lat_s'], region['lat_e']),
            lon=slice(region['lon_s'], region['lon_e']),
        )
        .weighted(np.cos(np.deg2rad(data.lat)))
        .mean(("lat", "lon"))
    )
    data.name = 'sssa_na'
    return data

region_na= dict(
    lat_s = 25,
    lat_e = 50,
    lon_s = -50,
    lon_e = -15
)

region_nan = dict(
    lat_s = 38,
    lat_e = 50,
    lon_s = -50,
    lon_e = -40
)

sssa_na = reg_mean(sssa_d, region_na, region_nan)

In [11]:
def reg_mean(data, region):

    data = (
        data.sel(
            lat=slice(region['lat_s'], region['lat_e']),
            lon=slice(region['lon_s'], region['lon_e']),
        )
        .weighted(np.cos(np.deg2rad(data.lat)))
        .mean(("lat", "lon"))
    )  
    data.name = 'sssa_sa'
    return data

region_sa = dict(
    lat_s = -22.5,
    lat_e = -10,
    lon_s = -42,
    lon_e = -10
)

sssa_sa = reg_mean(sssa_d, region_sa)

In [12]:
xr.merge([
    sssa_na,
    sssa_sa
]).reset_coords(names='depth', drop=True).to_netcdf("data/da_pred_salinty.nc")